## **Problem 5: Classification with KNN and Decision Trees**
**5.1-5.2**

In [142]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
dTrain = pd.read_csv('carseats_train.csv', names=None)
dTest = pd.read_csv('carseats_test.csv')

Data preprocessing function

In [143]:
def data_processing(data):
    dict_ShelveLoc = {'Bad': 0, 'Medium': 0.5, 'Good' : 1}
    dict_UrbanAndUS = {'No': 0, 'Yes' : 1}
    return    data.replace({**dict_ShelveLoc, **dict_UrbanAndUS})

Copying F1 function from Q3 and testing on tree_sklearn with different parameters.

In [144]:
def F1(labels_predicted, labels_true, print_values=True):
    TP, FP, TN, FN = ([0]*4)
    for pr, y in zip(labels_predicted, np.array(labels_true)):
        y = type(pr)(y)  # making sure that the types are the same
        if pr == 1:
            if pr == y:
                TP += 1
            else:
                FP += 1
        else:
            if pr == y:
                TN += 1
            else:
                FN += 1
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    if precision + recall != 0:
        F1 = 2 * (precision * recall) / (precision + recall)
    else:
        F1 = 0
    if print_values:
        print(f'Precision = {precision}, Recall = {recall}, F1 ={F1: 1.5f}')
    return F1

X = data_processing(dTrain.iloc[:, 1:])
y = dTrain['Sales']
X_test = data_processing(dTest.iloc[:, 1:])
y_test = dTest['Sales']
tree_sklearn = DecisionTreeClassifier(criterion='entropy', splitter='best',
                                     min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None,
                                     max_depth=None, max_features=None)
tree_sklearn.fit(X, y)
print('F1=', F1(tree_sklearn.predict(X_test), y_test))
tree_sklearn = DecisionTreeClassifier(criterion='entropy', splitter='best',
                                     min_samples_split=4, min_samples_leaf=1, max_leaf_nodes=8,
                                     max_depth=None, max_features=3)
tree_sklearn.fit(X, y)
print('F1=', F1(tree_sklearn.predict(X_test), y_test))
### Plotting the tree if we want ###
# text_representation = tree.export_text(tree_sklearn, feature_names=list(X.columns.values))
# print(text_representation)
# fig = plt.figure(figsize=(50,50))
# _ = tree.plot_tree(tree_sklearn, feature_names=X.columns.values, class_names=['Sold_1', 'Not_0'], filled=True)
# fig.savefig('.\\tree_sklearn.png')
# fig.show()

Precision = 0.7407407407407407, Recall = 0.6779661016949152, F1 = 0.70796
F1= 0.7079646017699114
Precision = 0.8125, Recall = 0.4406779661016949, F1 = 0.57143
F1= 0.5714285714285714


Testing chefboost package. F1 value. We see, that cheefbost strongly overfits data, so F1 values on test data
are kinda low.

In [145]:
from chefboost import Chefboost as chef
# otherwise chef can only use regression
dTrain_new =dTrain.astype({'Sales' : str})
dTest_new = dTest.astype({'Sales' : str})
# config = {'algorithm':'CART', 'enableGBM': True, 'epochs': 7, 'learning_rate': 1}
config = {'algorithm':'C4.5'}
for config in [{'algorithm':'C4.5'}, {'algorithm':'CART'}, {'algorithm':'CHAID'}]:
    # config = {'enableGBM': True, 'epochs': 7, 'learning_rate': 1, 'max_depth': 5}
    tree_chef = chef.fit(dTrain_new, config, target_label = 'Sales')
    # chef.evaluate(tree_chef, dTest_new, task='test', target_label = 'Sales')
    y_predicted = np.zeros(len(dTest_new))
    for i in range(len(dTest_new)):
        y_predicted[i] = chef.predict(tree_chef, param=(dTest_new.iloc[i, 1:]))
    print('F1=', F1(y_predicted, np.array(y_test)))

[INFO]:  8 CPU cores will be allocated in parallel running
C4.5  tree is going to be built...
-------------------------
finished in  5.521518707275391  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  79.43262411347517 % on  282  instances
Labels:  ['1' '0']
Confusion matrix:  [[55, 8], [50, 169]]
Precision:  87.3016 %, Recall:  52.381 %, F1:  65.4762 %
Precision = 0.8846153846153846, Recall = 0.3898305084745763, F1 = 0.54118
F1= 0.5411764705882353
[INFO]:  8 CPU cores will be allocated in parallel running
CART  tree is going to be built...
-------------------------
finished in  3.938645362854004  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  97.16312056737588 % on  282  instances
Labels:  ['1' '0']
Confusion matrix:  [[100, 3], [5, 174]]
Precision:  97.0874 %, Recall:  95.2381 %, F1:  96.1539 %
Precision = 0.6607142857142857, Recall = 0.6271186440677966, F1 = 0.64348
F1= 0.6434782608695652
[INFO]:  

K-Fold cross validation implementation

In [146]:
def K_Fold_cross_validation(method_predict, validation, X, y, K_values, K_name, fold_coeff=10, XTDEL=None, YTDEL=None, **config):
    # len_K - length of the fold_coeff-1 folds. The last one can be a little bit bigger to cover all data left
    len_K = len(X) // fold_coeff
    fullValidation = np.zeros((fold_coeff, len(K_values)))
    for i in range(fold_coeff - 1):
        XTraining = X.drop(X.index[len_K*i:len_K*(i+1)], axis=0)
        yTraining = y.drop(y.index[len_K*i:len_K*(i+1)], axis=0)
        XValid = X.iloc[len_K*i:len_K*(i+1), :]
        yValid = y.iloc[len_K*i:len_K*(i+1)]
        for j, K in enumerate(K_values):
            K_name_dict = {K_name : K}
            method = method_predict(**config, **K_name_dict)
            method.fit(XTraining, yTraining)
            fullValidation[i, j] = validation(method.predict(XValid), np.array(yValid), print_values=False)
    # last fold
    XTraining = X.drop(X.index[len_K*(fold_coeff - 1):], axis=0)
    yTraining = y.drop(y.index[len_K*(fold_coeff - 1):], axis=0)
    XValid = X.iloc[len_K*(fold_coeff - 1):, :]
    yValid = y.iloc[len_K*(fold_coeff - 1):]
    for j, K in enumerate(K_values):
        K_name_dict = {K_name : K}
        method = method_predict(**config, **K_name_dict)
        method.fit(XTraining, yTraining)
        fullValidation[fold_coeff - 1, j] = validation(method.predict(XValid), np.array(yValid), print_values=False)
    allKValidation = np.sum(fullValidation, axis=0) / fold_coeff
    print('mean K_Fold validation values')
    print(allKValidation)
    print(f'The best K value is # {np.argmax(allKValidation)}'
          f' with value K={K_values[np.argmax(allKValidation)]}')
    return allKValidation

Using K-Fold for the parameter 'max_features' for sklearn.tree.
10 Folds is used but it seems like it's too many for such small data. F1 values are too small, slightly above 0.5.


In [147]:
K_name = 'max_features'
K_values = [1,2, 3, 4, 5, 6, 7, 8, 9, 10]
fold_coeff = 10
method_predict = DecisionTreeClassifier
validation = F1
config = {'random_state': 10, 'criterion' : 'entropy', 'splitter' : 'best',
          'min_samples_split'  : 2, 'min_samples_leaf'  : 1,
          'max_leaf_nodes' : None}
kFold = K_Fold_cross_validation(method_predict, validation, X, y,
                                K_values, K_name, fold_coeff, X_test, y_test, **config)

# Training with the best 'max_features' parameter.
tree_sklearn = DecisionTreeClassifier(**config, max_features=K_values[np.argmax(kFold)])
tree_sklearn.fit(X, y)
print('F1=', F1(tree_sklearn.predict(X_test), y_test))

mean K_Fold validation values
[0.5260308  0.55724253 0.54691079 0.56041792 0.56930873 0.59578548
 0.57399338 0.50595559 0.56970607 0.53776525]
The best K value is # 5 with value K=6
Precision = 0.8, Recall = 0.7457627118644068, F1 = 0.77193
F1= 0.7719298245614035


Using K-Fold for the parameter 'algorithm' for Chefboost. Here I have to write a wrapper for Chefboost because it
works with the different data representation, compared to sklearn. 5 folds in work are showed  (K=5 since it's slow).

In [148]:
class chefMethod_helper():
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.fit = self.chefFit_helper
        self.predict = self.chefPredict_helper

    def chefFit_helper(self, X , y):
        Xy = pd.concat([y, X], axis=1)
        self.method = chef.fit(Xy, **self.kwargs)

    def chefPredict_helper(self, X):
        y_predicted = np.zeros(len(X))
        for i in range(len(X)):
            y_predicted[i] = chef.predict(self.method, param=(X.iloc[i, :]))
        return y_predicted

dTrain_new = dTrain.astype({'Sales' : str})
X = data_processing(dTrain_new.iloc[:, 1:])
y = dTrain_new['Sales']

K_name = 'config'
K_values = [{'algorithm':'ID3'}, {'algorithm':'C4.5'}, {'algorithm':'CART'}, {'algorithm':'CHAID'}]
fold_coeff = 5
method_predict = chefMethod_helper
validation = F1
config = {'target_label' : 'Sales'}
kFold = K_Fold_cross_validation(method_predict, validation, X, y,
                                K_values, K_name, fold_coeff, X_test, y_test, **config)

[INFO]:  8 CPU cores will be allocated in parallel running
ID3  tree is going to be built...
-------------------------
finished in  4.080012083053589  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  94.69026548672566 % on  226  instances
Labels:  ['1' '0']
Confusion matrix:  [[84, 11], [1, 130]]
Precision:  88.4211 %, Recall:  98.8235 %, F1:  93.3333 %
[INFO]:  8 CPU cores will be allocated in parallel running
C4.5  tree is going to be built...
-------------------------
finished in  3.637967586517334  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  78.76106194690266 % on  226  instances
Labels:  ['1' '0']
Confusion matrix:  [[44, 7], [41, 134]]
Precision:  86.2745 %, Recall:  51.7647 %, F1:  64.7059 %
[INFO]:  8 CPU cores will be allocated in parallel running
CART  tree is going to be built...
-------------------------
finished in  4.379726409912109  seconds
-------------------------
Evaluate  train s

Training with the best 'algorithm' parameter, which is CHAID according to K-Fold validation

In [149]:
tree_chef = chef.fit(dTrain_new, config = K_values[np.argmax(kFold)], target_label = 'Sales')
y_predicted = np.zeros(len(dTest_new))
for i in range(len(dTest_new)):
    y_predicted[i] = chef.predict(tree_chef, param=(dTest_new.iloc[i, 1:]))
print('F1=',F1(y_predicted, np.array(y_test)))

[INFO]:  8 CPU cores will be allocated in parallel running
CHAID  tree is going to be built...
-------------------------
finished in  6.160266876220703  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  92.19858156028369 % on  282  instances
Labels:  ['1' '0']
Confusion matrix:  [[92, 9], [13, 168]]
Precision:  91.0891 %, Recall:  87.619 %, F1:  89.3204 %
Precision = 0.6551724137931034, Recall = 0.6440677966101694, F1 = 0.64957
F1= 0.6495726495726496


**5.3** Implementation the KNN algorithm from scratch.

In [150]:
def distance_euclidean(x, y):
    xNp, yNp = np.array(x), np.array(y)
    return np.sqrt(np.sum((xNp - yNp) ** 2))

def distance_cosine(x, y):
    xNp, yNp = np.array(x), np.array(y)
    xDotY = np.sum(xNp * yNp)
    xMod = np.sqrt(np.sum(xNp ** 2))
    yMod = np.sqrt(np.sum(yNp ** 2))
    cosSimilarity = xDotY / xMod / yMod
    return 1 - cosSimilarity

def distance_from_array(x, Y, distance=distance_euclidean):
    return np.array([distance(x, y) for y in Y])

def normalization_array(array, return_coeff=True):
    arrayMod = np.sqrt(np.sum(array ** 2, axis=0))
    if return_coeff:
        return np.divide(array, arrayMod), arrayMod
    else:
        return np.divide(array, arrayMod)

def KNN(point, data, K,distance=distance_euclidean, label=None, binary=True):
    if label is None:
        label = data.columns[0]
    elif isinstance(label, int):
        label = data.columns[label]
    X = data.drop(label, axis=1).values
    y = data[label].values
    # normalization
    X, pointMod = normalization_array(X)
    point = point / pointMod
    dist = distance_from_array(point, X, distance=distance)
    indexDist = dist.argsort()[:K]
    # print(X[indexDist], y[indexDist])
    if binary:
        if y[indexDist].mean() >= 0.5:
            return 1
        else:
            return 0
    else:
        return y[indexDist].mean()

Testing our own KNN with 2 different distances: euclidean and cosine. Also different number of neighbors K.
The best performance for euclidean metrics achieved by setting K=4, for cosine with K=2.

In [151]:
def data_processing(data):
    dict_ShelveLoc = {'Bad': 0, 'Medium': 0.5, 'Good' : 1}
    dict_UrbanAndUS = {'No': 0, 'Yes' : 1}
    return    data.replace({**dict_ShelveLoc, **dict_UrbanAndUS})

dTrain = data_processing(pd.read_csv('carseats_train.csv', names=None))
dTest = data_processing(pd.read_csv('carseats_test.csv'))
points = dTest.iloc[:, 1:].values
# KNN(point, dTrain, 3, label=0)
y_predicted = np.zeros(len(dTest))
for distance in [distance_euclidean, distance_cosine]:
    print(f'Distance: {distance}')
    for K in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
        for i, point in enumerate(points):
            y_predicted[i] = KNN(point, dTrain, distance=distance, K=K, label=0)
        print(f'K={K}, F1={F1(y_predicted, np.array(y_test), print_values=False)}')

Distance: <function distance_euclidean at 0x0000018D17B18F70>
K=1, F1=0.613861386138614
K=2, F1=0.7086614173228345
K=3, F1=0.673076923076923
K=4, F1=0.7377049180327869
K=5, F1=0.6464646464646464
K=6, F1=0.6909090909090908
K=7, F1=0.6999999999999998
K=8, F1=0.7037037037037036
K=9, F1=0.6804123711340205
K=10, F1=0.6915887850467289
K=11, F1=0.6999999999999998
Distance: <function distance_cosine at 0x0000018D18592C10>
K=1, F1=0.607843137254902
K=2, F1=0.732824427480916
K=3, F1=0.7047619047619047
K=4, F1=0.7130434782608696
K=5, F1=0.7047619047619047
K=6, F1=0.6956521739130435
K=7, F1=0.6990291262135923
K=8, F1=0.6788990825688074
K=9, F1=0.6990291262135923
K=10, F1=0.7027027027027027
K=11, F1=0.6923076923076924


**5.4**
(all the evaluations are based on F1 values)
The worst performance is shown by Chefboost implementation of decision trees. I could not get a high
performance even trying all the available algorithms and I haven't found the way to adjust other parameters.
Seems like it overfit a lot.  F1<0.7.
It's also the slowest one.

The best performance was reached by sklearn.tree.DecisionTreeClassifier with "max_features"=5. F1=0.77

KNN algorithm is close to sklearn.tree but it's still performs worse: F1=0.74. (But I believe, with some
weights tuning it can do better)

**5.5**
In K-fold cross-validation we divide our data into K "almost" equally sized subsets (where people often use K=10)
and repeat training/testing model K times, each time rotation the test fold. However, in  Leave-One-Out
cross-calidation our test fold consists of only 1 data sample and the rest of the data is used for training.
This process is repeated N times (where N is the number of data points) and the "test fold" is rotating
through all the data points. Basically, it's K-fold with K=N.

Disadvantages of the LOOCV:

1) If the data is big, it's very time and resource consuming process since it requires N trainings for your algorithm.

2) LOOCV can overfit a lot since it's trained on almost all the data and is being validated on only 1 sample
(as a result it can also have a very high variability in predictions)

**5.6**
I believe, that F1 is a good metric for that problem (especially compared to Accuracy) because the data is
quite imbalance (there are almost 2 times less sales).

F1 fully confirmed the "feeling" (described in **5.4**: scrlen the best, KNN 2nd, Chefboost the worst)
I had about all the algorithms I was studying in this problem.

In [ ]:
standBy = True
While True:
    if standBy:
        if спец жест:
            standBy = False
    else:
        do everything
        if спец жест:
            standBy = True
            continue